In [1]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('AI_Human.csv')
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [3]:
df['generated'].value_counts()

generated
0.0    305797
1.0    181438
Name: count, dtype: int64

In [37]:
df['text'][1]

'Transportation is a large necessity in most countries worldwide. With no doubt, cars, buses, and other means of transportation make going from place to place easier and faster. However there\'s always a negative pollution. Although mobile transportation are a huge part of daily lives, we are endangering the Earth with harmful greenhouse gases, which could be suppressed.\n\nA small suburb community in Germany called Vauban, has started a "carfree" lifestyle. In this city, markets and stores are placed nearby homes, instead of being located by farend highways. Although Vauban is not completely carfree, 70% of Vauban families do not own cars Even a large 57% of families stated to have sold their cars to move to Vauban. Some families have even said to be less stressed depending on car transportation. Cars are responsible for about 12% of greenhouse gases, and can even be up to 50% in some carintensive areas in the United States.\n\nAnother insight to reduced car zones brings Paris\' incid

In [4]:
df["text_length"] = df["text"].apply(len)
human_stats = df[df["generated"] == 0]["text_length"].describe()
ai_stats = df[df["generated"] == 1]["text_length"].describe()

In [5]:
human_stats

count    305797.000000
mean       2354.592838
std        1082.013088
min          82.000000
25%        1543.000000
50%        2156.000000
75%        2920.000000
max       18322.000000
Name: text_length, dtype: float64

In [6]:
ai_stats

count    181438.000000
mean       2126.316411
std         787.254863
min           1.000000
25%        1654.000000
50%        2048.000000
75%        2480.000000
max        8871.000000
Name: text_length, dtype: float64

In [7]:
df.shape

(487235, 3)

In [8]:
x = df['text']
y = df['generated']

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [10]:
x_train.shape

(389788,)

In [11]:
x_test.shape

(97447,)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.95, min_df=5, ngram_range=(1,2), max_features=100000,dtype=np.float64)

In [13]:
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [14]:
y_train.value_counts()

generated
0.0    244685
1.0    145103
Name: count, dtype: int64

In [15]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(
    alpha=10
)
mnb.fit(x_train, y_train)

MultinomialNB(alpha=10)

In [16]:
mnb.score(x_test, y_test)

0.9674284482847086

In [ ]:
lr = LogisticRegression(C=10, max_iter=1000)
lr.fit(x_train, y_train)

In [29]:
lr.score(x_test, y_test)

0.9972395250751691

In [100]:
sentence = 'lorem ipsum'

In [ ]:
words = sentence.s

In [101]:
y_pred = vectorizer.transform([sentence])

In [84]:
ans = mnb.predict(y_pred)
ans

array([0.])

In [85]:
proba_mnb = mnb.predict_proba(y_pred)[0]

In [86]:
from sklearn.linear_model import LogisticRegression

In [87]:
proba_lr = lr.predict_proba(y_pred)[0]

In [88]:
proba = proba_mnb*0.75+proba_lr*0.25

In [89]:
if proba[0] > proba[1]:
    print(0)
else:
    print(1)

1


In [81]:
proba

array([0.74606572, 0.25393428])

In [62]:
import pickle
with open('mnb_model.pkl', 'wb') as file:
    pickle.dump(mnb, file)

In [82]:
with open('lr_model.pkl', 'wb') as file:
    pickle.dump(lr, file)

In [63]:
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [99]:
import shap
import numpy as np

# Reduce sample size significantly
num_train_samples = 50  # Reduce further to prevent memory errors
num_test_samples = 3  # Use even fewer test samples

x_train_sample = x_train[np.random.choice(x_train.shape[0], num_train_samples, replace=False)].toarray()
x_test_sample = x_test[:num_test_samples].toarray()

# Use SHAP's sampling-based explainer for Naive Bayes
explainer_mnb = shap.KernelExplainer(mnb.predict_proba, shap.sample(x_train_sample, 10))  # 10 samples only
shap_values_mnb = explainer_mnb.shap_values(x_test_sample)

# Use SHAP's efficient LinearExplainer for Logistic Regression
explainer_lr = shap.LinearExplainer(lr, x_train_sample)
shap_values_lr = explainer_lr(x_test_sample)

# Weighted SHAP value combination
shap_values_avg = (np.array(shap_values_mnb) * 0.55) + (np.array(shap_values_lr) * 0.45)

# Plot results
shap.summary_plot(shap_values_avg, x_test_sample, feature_names=vectorizer.get_feature_names_out())


  0%|          | 0/3 [00:00<?, ?it/s]

MemoryError: Unable to allocate 63.5 GiB for an array with shape (8528, 1000000) and data type float64